Executive Summary:

The goal of this project is to predict precipitation in Texas at time point “t+1” given temperature at time “t”. This analysis uses CPC gauge-based gridded daily precipitation and daily temperature data over roughly the area of Texas from the years 2000 to 2009 to train two different predictive models. Both models applied a principal component analysis (PCA) to downscale the high dimensional temperature data, and then fit models on the selected principal components. In the first approach, K-nearest neighbors (KNN) was applied to the principal components to predict precipitation based on a subsetted testing set of temperature data to allow for comparison to actual values. In the second approach, a linear regression model was applied to the first principal component only to predict precipitation corresponding to the same testing temperature dataset. 

In order to validate the fit of the models, mean squared error, mean absolute error, and residuals were calculated. These metrics were also used to compare the performance between the two models. Though upon visual inspection PCA-KNN appears to predict precipitation trends closer to observed values, the PCA-KNN approach exhibited a higher mean squared error and mean absolute error than the PCA-linear regression model despite a lower average residual value. Higher mean absolute error and mean squared error indicates that the PCA-KNN model actually performs worse than the PCA-linear regression model, likely due to large variations and inaccuracies in extreme values. The PCA-linear regression model, on the other hand, exhibits a conservative, “dreary” prediction across all time points, which leads to lower error overall when dealing with extremes.

Methods

Data Management/Preprocessing
The original precipitation data was subsetted to only include the years 2000 to 2009 to correspond to the selected temperature data timeframe. Temperature data was subsetted to match the space occupied by the precipitation data and cover roughly the area of Texas. Latitude and longitude were adjusted to ensure consistent formatting across the datasets. To predict on precipitation at time t+1, the precipitation time points used in the analysis were shifted one day from the temperature data time points. Precipitation and temperature data were then split to create training and testing sets, with years 2000 to 2007 included in training and the rest allocated to testing. Temperature data was preprocessed by calculating the mean climatology and obtaining the anomalies rather than working with raw temperatures to better account for seasonal variation.

1. Setup

1.1 Load Packages


In [ ]:
using Dates
using MultivariateStats
using Plots
using NCDatasets
using StatsBase
using Unitful
using Distances
using GLM
using NetCDF
using DataFrames

2. Data

2.1 Precipitation
2.1.1 Load data


In [ ]:
#load precip data from NCDataset, this precip is over the area of Texas
precip_ds = NCDataset("data/raw/precip_tx.nc")
#define time, longitude, latitude, and precip as variables. precip is a matrix and time, lon, and lat are vectors
precip_time = precip_ds["time"][:]
precip_lon = precip_ds["lon"][:]
precip_lat = precip_ds["lat"][:]
precip = precip_ds["precip"][:,:,:]

2.1.2 Close precip dataset


In [ ]:
close(precip_ds) 

2.1.3 Filter data


In [ ]:
#filter precip to only include 2000-2009
t_start = 2000
t_end = 2009

start_ind = findfirst(date -> year(date) >= t_start, precip_time)
end_ind = findlast(date -> year(date) <= t_end, precip_time)
precip = precip[:,:, start_ind:end_ind]
precip_time = precip_time[start_ind:end_ind]

2.1.4 Subset data


In [ ]:
#subset precip to time range t+1
t_start = Dates.DateTime(2000, 1, 2)
t_end = Dates.DateTime(2009, 12, 31)

#create index for filtering 

precip_indices = t_start .<= precip_time .<= t_end

precip = precip[:, :, precip_indices]
precip_time_plus = precip_time[precip_indices]

2.1.5 Reverse latitude


In [ ]:
#dims 2 is telling to reverse along the second dimension (latitude)
precip_lat = reverse(precip_lat)
precip = reverse(precip;dims=2)

2.1.6 Convert lon to lon1 format


In [ ]:
#Convert long3 to long1 format (source: https://confluence.ecmwf.int/pages/viewpage.action?pageId=149337515)
function convert_longitude(longitude)
    lon1 = ifelse(longitude > 180, longitude - 360, longitude)
    return lon1
end

precip_lon = convert_longitude.(precip_lon)

2.2 Temperature

2.2.1 Load data
#(move to get_data.jl file)?


In [ ]:
#This code block was written by Chatgpt to resample hourly temperature data to daily temperature data

# Write a function to convert hourly temperature to daily average
function resample_to_daily_average(temp, temp_time)

    #use floor to extract the date from the time, save as date_vector
    date_vector = floor.(temp_time, Dates.Day)
    #find all unique dates in the date_vector, save as unique_dates
    unique_dates = unique(date_vector)

    # Find the size of the temperature array, save as lon_points and lat_points
    lon_points, lat_points, _ = size(temp)

    # Create an empty array to store daily temperature averages
    daily_temps = []

    # Calculate daily mean by grouping unique dates
    for date in unique_dates
        # Find indices of all time points under a unique date
        indices = findall(date_vector .== date)
        
        # Calculate daily mean for each lonxlat point
        daily_mean = mean(temp[:, :, indices], dims=3)[:, :, 1]
        
        # Add daily mean to the empty array
        push!(daily_temps, daily_mean)
    end

    # Concatenate daily means into a 3D array along the 3rd dimension
    daily_temps = cat(daily_temps..., dims=3)

    return daily_temps
end

#Write a function to call and process each year of temperature data
function process_year(data_path)
    # Extract year from filename, all files follow a common path structure
    match_result = match(r"2m_temperature_(\d{4})\.nc", data_path)
    #save individual years as string
    year_str = match_result.captures[1]
    #save as variable of integers
    year = parse(Int, year_str)

    # Load year's data, all files follow common data_path
    temp_dataset = NCDataset(data_path)

    # Save lon, lat, time, and temperature as variables
    longitude = temp_dataset["longitude"][:]
    latitude = temp_dataset["latitude"][:]
    time = temp_dataset["time"][:]
    t2m = temp_dataset["t2m"][:, :, :]

    #save time as temp_time
    temp_time = time

    # Resample temperature data to daily averages
    daily_temps_result = resample_to_daily_average(t2m, temp_time)

    return year, daily_temps_result
end

# Paths to your data files for each year
data_paths = [
    "data/raw/2m_temperature_2000.nc",
    "data/raw/2m_temperature_2001.nc",
    "data/raw/2m_temperature_2002.nc",
    "data/raw/2m_temperature_2003.nc",
    "data/raw/2m_temperature_2004.nc",
    "data/raw/2m_temperature_2005.nc",
    "data/raw/2m_temperature_2006.nc",
    "data/raw/2m_temperature_2007.nc",
    "data/raw/2m_temperature_2008.nc",
    "data/raw/2m_temperature_2009.nc"
]

# Process each year and combine the results
all_years_temps = Dict(process_year(path) for path in data_paths if process_year(path) !== nothing)

2.2.2 Combine


In [ ]:
#combine all years into one array
#call all years and save each as a matrix with lon = 66, lat = 27, and time in days, 365 for non-leap years and 366 for leap years
temp_2000 = reshape(all_years_temps[2000], 66, 27, 366)
temp_2001 = reshape(all_years_temps[2001], 66, 27, 365)
temp_2002 = reshape(all_years_temps[2002], 66, 27, 365)
temp_2003 = reshape(all_years_temps[2003], 66, 27, 365)
temp_2004 = reshape(all_years_temps[2004], 66, 27, 366)
temp_2005 = reshape(all_years_temps[2005], 66, 27, 365)
temp_2006 = reshape(all_years_temps[2006], 66, 27, 365)
temp_2007 = reshape(all_years_temps[2007], 66, 27, 365)
temp_2008 = reshape(all_years_temps[2008], 66, 27, 366)
temp_2009 = reshape(all_years_temps[2009], 66, 27, 365)

#combine 
temp = cat(temp_2000, temp_2001, temp_2002, temp_2003, temp_2004, temp_2005, temp_2006, temp_2007, temp_2008, temp_2009, dims=3)

2.2.3 Save lon, lat, time, and temp variables


In [ ]:
#Having trouble writing variable using same form of calling above, used data_dict instead but not optimal

#save data_dict for 2000
data_dict = open_mfdataset(["data/raw/2m_temperature_2000.nc"], "t2m")

#define lon and lat based on year 2000 (should be the same for all years) and temp_time as precip_time (should be the same)
temp_lon = data_dict["longitude"][:]
temp_lat = data_dict["latitude"][:]
temp_time = precip_time

2.2.4 Flip the temperature latitude


In [ ]:
temp_lat = reverse(temp_lat)
temp = reverse(temp;dims=2)

2.2.5 Subset temperature to area of Texas instead of US


In [ ]:
# Define longitude and latitude ranges for filtering based on precip data which is already filtered to Texas
lon_min = minimum(precip_lon)
lon_max = maximum(precip_lon)
lat_min = minimum(precip_lat)
lat_max = maximum(precip_lat)

# Create masks for latitude and longitude based on specified ranges
lat_mask = (lat_min .<= temp_lat .<= lat_max)
lon_mask = (lon_min .<= temp_lon .<= lon_max)

# Convert BitVector to Vector{Int} using findall
lat_indices = findall(lat_mask)
lon_indices = findall(lon_mask)

# Update latitude and longitude data variables
temp_lat = temp_lat[lat_indices]
temp_lon = temp_lon[lon_indices]

# Subset temperature data based on latitude and longitude indices
temp = temp[lon_indices, lat_indices, :]

2.2.6 Subset temperature time


In [ ]:
#subset temperature to time range t 
t_start = Dates.DateTime(2000, 1, 1)
t_end = Dates.DateTime(2009, 12, 30)

#Create index for filtering
temp_indices = t_start .<= temp_time .<= t_end

temp = temp[:, :, temp_indices]

temp_time = temp_time[temp_indices]

2.3 Split data into training and testing sets


In [ ]:
#2000-2007 for training, 2008-2009 for testing
#Define start and end times
temp_t_start = Dates.DateTime(2000, 1, 1)
temp_t_end = Dates.DateTime(2007, 12, 30)

# Create indices for temperature
temp_train_indices = temp_t_start .<= temp_time .<= temp_t_end
temp_test_indices = temp_time .> temp_t_end

#Subset temperature data based on indices
temp_train = temp[:, :, temp_train_indices]
temp_test = temp[:, :, temp_test_indices]

#Subset temperature time based on indices
time_train = filter(date -> temp_t_start <= date <= temp_t_end, temp_time)
time_test = filter(date -> temp_t_end <= date , temp_time)

#Create indices for precipitation
precip_t_start = Dates.DateTime(2000, 1, 2)
precip_t_end = Dates.DateTime(2007, 12, 31)

#Subset precipitation data based on indices
precip_train_indices = precip_t_start .<= precip_time_plus .<= precip_t_end
precip_test_indices = precip_time_plus .> precip_t_end

#Subset precipitation data based on indices
precip_train = precip[:, :, precip_train_indices]
precip_test = precip[:, :, precip_test_indices]


2.4 Preprocessing

2.4.1 Preprocess function


In [ ]:
function preprocess(temp::Array{T, 3}, temp_ref::Array{T,3}) where T
  #we use temp_ref because we want the climatology to be the same between the test and train datasets!
           #name lon, lat, and time as the dimensions of the temp input matrix
           lon, lat, time = size(temp)
           #find the climatology, dims=3 says take along the 3rd dimension (time)
           climatology = mean(temp_ref, dims=3)
           temp_anom = temp .- climatology
           #reshape the data by multiplying lonxlat to a 2D matrix
           temp_anom = reshape(temp_anom, (lon * lat, time))
           #return
           return temp_anom
       end

2.4.2 Preprocess temperature


In [ ]:
temp_train_proc = preprocess(temp_train, temp_train)
temp_test_proc = preprocess(temp_test, temp_train)

2.4.3 Reshape precip


In [ ]:
#reshape precip_train and precip_test to 2D matrices
precip_train_reshape = reshape(precip_train, (24 * 24, 2921))
precip_test_reshape = reshape(precip_test, (24 * 24, 731))

3. Principal Component Analysis
3.1 Fit PCA model


In [ ]:
pca_model = fit(PCA, temp_train_proc; maxoutdim=25);

3.2 Plot variance to determine number of PCs to keep


In [ ]:
#plot the variance explained
var_explained = plot(
    principalvars(pca_model) / var(pca_model),
    xlabel="PC",
    ylabel="Fraction of variance explained",
    label=false,
    title="Variance Explained by PCs"
)
#plot the cumulative variance explained (cdf)
cum_var_explained = plot(
    cumsum(principalvars(pca_model)) / var(pca_model);
    xlabel="PC",
    ylabel="Fraction of variance explained",
    label=false,
    title="Cumulative Variance Explained"
)
#Plot 
plot(var_explained, cum_var_explained, layout=(1,2), size=(800,400))

3.3 Transform PCs


In [ ]:
#transform temp_train onto PCA space
temp_train_transform = transform(pca_model, temp_train_proc)
#predict temp_test 
temp_test_transform = predict(pca_model, temp_test_proc)

3.4 Save first three PCs


In [ ]:
PC1 = temp_train_transform[1,:]
PC2 = temp_train_transform[2,:]
PC3 = temp_train_transform[3,:]

3.5 Plot PCA

3.5.1 Plot time series of first three PCs


In [ ]:
pc1_plot = scatter(time_train, PC1, label="PC1", xlabel="Time", ylabel="PC1", title="PC1 Time Series", legend=:topleft)
pc2_plot = scatter(time_train, PC2, label="PC2", xlabel="Time", ylabel="PC2", title="PC2 Time Series", legend=:topleft)
pc3_plot = scatter(time_train, PC3, label="PC3", xlabel="Time", ylabel="PC3", title="PC3 Time Series", legend=:topleft)

3.5.2 Plot first 2 PCs and mean precipitation


In [ ]:
#plot the first two PCs with the mean precipitation
#replace missing with 0.0
precip_train_zero = coalesce.(precip_train_reshape, 0.0)
replace!(precip_train_zero, NaN => 0.0)
mean_precip = mean.(precip_train_zero)

pc_heat = scatter(PC1, PC2, zcolor=mean_precip'; xlabel="PC1", ylabel="PC2", title="PC1 vs PC2", legend=:topleft)

3.5.3 Plot the second and third PCs with mean precipitation


In [ ]:
pc_heat = scatter(PC2, PC3, zcolor=mean_precip'; xlabel="PC1", ylabel="PC2", title="PC2 vs PC3", legend=:topleft)

4. Approach 1: KNN
	
Given the high dimensional structure of the gridded temperature data, a principal components analysis was performed to downscale the features while preserving variation to allow for modeling across space and time. PCA effectively reduces the number of columns (in this case, locations) by projecting the data onto new PC axes. The number of principal components to retain was determined by plotting the variance explained by the principal components and the cumulative variance. After analyzing these figures, a break in variance is observed after principal component number two. The cumulative variance explained plot reveals that approximately 97.8% of the variance is explained by the first three principal components, so three principal components were retained for further analysis. The PCA model was fit to the training temperature data and then the temperature training and testing data were transformed onto the PCA space.

KNN was then used to predict the precipitation data for the testing temperature data given temperature and precipitation over the training period.The KNN function calculates the Euclidean distance between the new datapoint and the existing K data points and assigns weights based on these distances. Weighted sampling is then conducted to obtain the predicted value based on the indexed position sampled. Three was chosen as the hyperparameter for the number of neighbors because this number of neighbors is standard in KNN analysis and serves as an appropriate baseline. The resultant predicted precipitation values were compared to the actual precipitation training data to evaluate the fit of the model using residuals, MAE, and MSE.


4.1 Define KNN function


In [ ]:
#source: lab 6
#define euclidean distance function
function euclidean_distance(x::AbstractVector, y::AbstractVector)::AbstractFloat
return sqrt(sum((x .- y) .^ 2))
end

#define nsmallest function
function nsmallest(x::AbstractVector, n::Int)::Vector{Int}
idx = sortperm(x)
return idx[1:n]
end

#define knn function
function knn(X::AbstractMatrix, X_i::AbstractVector, K::Int)::Tuple{Int,AbstractVector}
# calculate the distances between X_i and each row of X
dist = [euclidean_distance(X_i, X[j, :]) for j in 1:size(X, 1)]
idx = nsmallest(dist, K)
w = 1 ./ dist[idx]
w ./= sum(w)
idx_sample = sample(idx, Weights(w))
return (idx_sample, vec(X[idx_sample, :]))
end

4.2 Combining PCA and KNN


In [ ]:
#source: lab 6
function predict_knn(
                  temp_train::Array{Union{Missing, Float64}, 3}, 
                  temp_test::Array{Union{Missing, Float64}, 3}, 
                  precip_train::Array{AbstractFloat, 3},
                  n_pca::Int64, 
                  K::Int64)
    #preprocessing
        temp_train_proc = preprocess(temp_train, temp_train) 
        temp_test_proc = preprocess(temp_test, temp_test) 
    #dimensionality reduction using PCA
        #apply PCA to test
        pca_model = fit(PCA, temp_train_proc; maxoutdim=n_pca)
        #predict test data onto PCA
        predict_train = predict(pca_model, temp_train_proc)
        predict_test = predict(pca_model, temp_test_proc)
              
    # use the `knn` function for each point in the test data
        predicted_precip = map(1:size(temp_test_proc, 2)) do i
        #find the indices of the n smallest values in x
        #K = number of nearest neighbors
        index, _ = knn(predict_train', predict_test[:,i], K)
                      precip_train[:,:, index]
         end
                 
   
    return predicted_precip
end

4.3 Test the model


In [ ]:
#replace missing with 0.0
precip_train_zero = coalesce.(precip_train, 0.0)
#predict test precipitation 
pca_knn_test_predict = predict_knn(temp_train, temp_test, precip_train_zero, 3, 3 )
#convert vector of matrices to 3D array
pca_knn_test_predict_reshape = cat(pca_knn_test_predict..., dims=3)

4.4 Analyze Fit

4.4.1 Reshape data for plotting


In [ ]:
#replace missing with 0.0
precip_test_zero = coalesce.(precip_test, 0.0)

#rehape to 2D array
precip_test_reshape = reshape(precip_test_zero, (24*24, 731))
pca_knn_test_predict_2D = reshape(pca_knn_test_predict_reshape, (24*24, 731))

4.4.2 Time series actual vs predicted test precipitation


In [ ]:
#plot predicted precipitation vs actual precipitation at a single grid cell
plot(time_test[1:731], pca_knn_test_predict_2D[1, :], label="Actual Precipitation", xlabel="Time", ylabel="Precipitation", title="Precipitation at t+1 Predicted by KNN- GC 1", legend=:topleft)
plot!(time_test[1:731], precip_test_reshape[1,:], label="Predicted Precipitation", xlabel="Time", ylabel="Precipitation", legend=:topleft)

In [ ]:
#plot predicted precipitation vs actual precipitation at a single grid cell
plot(time_test[1:731], pca_knn_test_predict_2D[150, :], label="Actual Precipitation", xlabel="Time", ylabel="Precipitation", title="Precipitation at t+1 Predicted by KNN- GC 150", legend=:topleft)
plot!(time_test[1:731], precip_test_reshape[150,:], label="Predicted Precipitation", xlabel="Time", ylabel="Precipitation", legend=:topleft)

In [ ]:
#plot predicted precipitation vs actual precipitation at a single grid cell
plot(time_test[1:731], pca_knn_test_predict_2D[400, :], label="Actual Precipitation", xlabel="Time", ylabel="Precipitation", title="Precipitation at t+1 Predicted by KNN- GC 400", legend=:topleft)
plot!(time_test[1:731], precip_test_reshape[400,:], label="Predicted Precipitation", xlabel="Time", ylabel="Precipitation", legend=:topleft)

4.4.3 Heatmaps


In [ ]:
#Plot for several time points at all grid cells
knn_pred_hm_1 = heatmap(pca_knn_test_predict_reshape[:,:,1]; xlabel="Longitude", ylabel="Latitude", title="Predicted Precipitation at t+1 Predicted by KNN", legend=:topleft)

knn_act_hm_1 = heatmap( precip_test_zero[:, :, 1]; label="Actual Precipitation", xlabel="Longitude", ylabel="Latitude", title="Actual Precipitation at t+1 Predicted by KNN", legend=:topleft)

knn_pred_hm_150 = heatmap(pca_knn_test_predict_reshape[:,:,150]; xlabel="Longitude", ylabel="Latitude", title="Predicted Precipitation at t+1 Predicted by KNN", legend=:topleft)


knn_act_hm_150 = heatmap( precip_test_zero[:, :, 150]; label="Actual Precipitation", xlabel="Longitude", ylabel="Latitude", title="Actual Precipitation at t+1 Predicted by KNN", legend=:topleft)

knn_pred_hm_400 = heatmap(pca_knn_test_predict_reshape[:,:,400]; xlabel="Longitude", ylabel="Latitude", title="Predicted Precipitation at t+1 Predicted by KNN", legend=:topleft)


knn_act_hm_400 = heatmap( precip_test_zero[:, :, 400]; label="Actual Precipitation", xlabel="Longitude", ylabel="Latitude", title="Actual Precipitation at t+1 Predicted by KNN", legend=:topleft)

4.4.4 Evaluate fit using MSE


In [ ]:
#Define mse function
function mean_se(x::AbstractVector, y::AbstractVector)::AbstractFloat
return mean((x .- y) .^ 2)
end

In [ ]:
# Calculate mse for each point in precip_test
#create an empty vector
mse_knn = []
for i in 1:size(precip_test_zero, 1)
    mse_ = mean_se(pca_knn_test_predict_2D[:,i], precip_test_zero[:,i])
    #store mse in vector
    push!(mse_knn, mse_)
end

4.4.5 Evaluate fit using MAE


In [ ]:
#Define mae function
function mean_abs_error(x::AbstractVector, y::AbstractVector)::AbstractFloat
return mean(abs.(x .- y))
end

In [ ]:
#create empty vector
mae_knn = []
for i in 1:size(precip_test_zero, 1)
    mae_ = mean_abs_error(pca_knn_test_predict_2D[:,i], precip_test_zero[:,i])
    #store mae in vector
    push!(mae_knn, mae_)
end

4.4.6 Evaluate fit using residuals


In [ ]:
#Define residuals function
function residuals(x::AbstractVector, y::AbstractVector)::AbstractVector
return x .- y
end

In [ ]:
#create vector for residuals
residuals_knn = Vector{Float32}(undef, size(precip_test_zero, 1) * size(precip_test_zero, 2))

k = 1
for i in 1:size(precip_test_zero, 1)
    for j in 1:size(precip_test_zero, 2)
        residuals_knn[k] = pca_knn_test_predict_2D[i, j] - precip_test_zero[i, j]
        k += 1
    end
end

5. Approach 2: PCA/Linear Regression

Similarly to Approach 1, Approach 2 employed PCA to reduce the dimensions of the temperature data. Only the first principal component was retained for further linear regression to allow for the creation of simple vectors that are compatible with linear regression syntax. Linear regression was performed on precipitation at each grid cell location following the first principal component of the temperature data. The temperature test data was then applied to this linear model to predict precipitation and was similarly compared to actual observed precipitation to evaluate the model using residuals, MAE, and MSE.

5.1 Linear Regression


In [ ]:
#fit the PCA model to training data and transform the temp train and testing data onto the PCA space
pca_model = fit(PCA, temp_train_proc; maxoutdim=1)
                  #predict test data onto PCA
                  predict_train = predict(pca_model, temp_train_proc)
                  predict_test = predict(pca_model, temp_test_proc)

#transpose predict_train and predict_test
temp_train_transform = predict_train'
temp_test_transform = predict_test'
#save as vectors
temp_train_transform = temp_train_transform[:]
temp_test_transform = temp_test_transform[:]

#remove missing values from precip_train
precip_train_zero = coalesce.(precip_train, 0.0)
#reshape to lonxlat
precip_train_reshape = reshape(precip_train_zero, (24 * 24, 2921))

In [ ]:
#regression for all grid cells in precip_train
# Create an empty vector to store predictions
all_predictions = Vector{Vector{Float64}}()
for grid_cell in 1:size(precip_train_reshape, 1)
    precip_train_linear = convert(Vector{Float64},precip_train_reshape[grid_cell, :])
    # Create a DataFrame for the training data
    df = DataFrame(temp_train_transform = temp_train_transform, precip_train_linear = precip_train_linear)

    # Fit linear regression model
    linear_model = lm(@formula(precip_train_linear ~ temp_train_transform), df)

    # Create a DataFrame for the test data
    df_test = DataFrame(temp_train_transform = temp_test_transform)

    # Predict precipitation for temp_test
    y_pred = predict(linear_model, df_test)
 push!(all_predictions, y_pred)
end
#convert to matrix
all_predictions_matrix = hcat(all_predictions...)

5.2 Analyze Fit
5.2.1 Plot time series of actual vs predicted precipitation at different grid cells


In [ ]:
#plot y_pred vs test_time at different grid cells
plot(time_test[1:731], all_predictions_matrix[:,1], label="Predicted Precipitation", xlabel="Time", ylabel="Precipitation", title="Linear Regression", legend=:topleft)
plot!(time_test[1:731], precip_test_reshape[1, :], label="Actual Precipitation", xlabel="Time", ylabel="Precipitation", title="Precipitation at t+1 Predicted by Lin Reg- GC 1", legend=:topleft)

In [ ]:
plot(time_test[1:731], all_predictions_matrix[:,150], label="Predicted Precipitation", xlabel="Time", ylabel="Precipitation", title="Linear Regression", legend=:topleft)
plot!(time_test[1:731], precip_test_reshape[150, :], label="Actual Precipitation", xlabel="Time", ylabel="Precipitation", title="Precipitation at t+1 Predicted by Lin Reg- GC 150", legend=:topleft)

In [ ]:
plot(time_test[1:731], all_predictions_matrix[:,400], label="Predicted Precipitation", xlabel="Time", ylabel="Precipitation", title="Linear Regression", legend=:topleft)
plot!(time_test[1:731], precip_test_reshape[400, :], label="Actual Precipitation", xlabel="Time", ylabel="Precipitation", title="Precipitation at t+1 Predicted by Linear Regression- GC 400", legend=:topleft)

5.2.2 Evaluate Fit using MSE


In [ ]:
mse_lin = []
for i in 1:size(precip_test_reshape, 1)
    mse_ = mean_se(all_predictions_matrix[i,:], precip_test_reshape[:,i])
    push!(mse_lin, mse_)
end

5.2.3 Evaluate fit using MAE


In [ ]:
mae_lin = []
for i in 1:size(precip_test_reshape, 1)
    mae_ = mean_abs_error(all_predictions_matrix[i,:], precip_test_reshape[:,i])
    push!(mae_lin, mae_)
end

5.2.4 Evaluate fit using residuals


In [ ]:
residuals_lin = Vector{Float32}(undef, size(precip_test_reshape, 1) * size(precip_test_zero, 2))

k = 1
for i in 1:size(precip_test_reshape, 1)
    for j in 1:size(precip_test_reshape, 2)
        residuals_lin[k] = all_predictions_matrix[j, i] - precip_test_reshape[i,j ]
        k += 1
    end
end

6. Compare

When plotting the predicted precipitation and the actual precipitation for the test data, upon visual inspection it seems that the PCA-KNN model more closely resembles the trends of the actual precipitation. The PCA-KNN predicted precipitation of varying levels while the PCA-linear regression model predicted nearly constant, low-level precipitation year-round, with some reflection of the seasonal trends as the maximum periods of the curves occur over the same range of time. However, upon closer analysis, the PCA-KNN model predictions differed significantly from the actual precipitation values. When considering only the residuals of actual precipitation minus predicted precipitation, the PCA-KNN model appears to perform better because the average of the residuals is lower than the PCA-linear regression model. However, when considering the mean absolute error and the mean squared error, the PCA-linear regression model outperformed the PCA-KNN model with significantly lower MAE and MSE, indicating overall that predictions are closer to actual values than the PCA-KNN model. Residuals might be lower than MSE or MAE for a model if there are both undershoots and overshoots when considering the entire dataset of predictions. For example, if the PCA-KNN model predicted 0 mm on a day that should be 40 mm and 40 mm on a day that should be 0 mm, the averaged residuals would be zero since they cancel out. MSE and MAE, however, are a more robust measure of model performance because they account for differences in sign via squaring and absolute value respectively. MSE places a greater emphasis on large errors but is sensitive to outliers, which explains why the difference in MSE is greater between the two models than MAE. 

The PCA-KNN model appears to be capturing the general shape of the data better, but local predictions are not very reliable based on the MAE and MSE calculations. This model could be further optimized by optimizing the hyperparameters n_pca and K. N_pca is the number of principal components retained in the model. Three was selected as the n_pca value in this analysis based on fraction of variance explained, but a systematic approach could be taken to test all values of PCs and retain the number with the lowest MSE/MAE. A similar systematic approach could be used to optimize the K parameter as well. 

The PCA-linear regression model is demonstrating a “dreary” effect by predicting low level precipitation across the time series. While there are periods of slight increase and decrease over the year corresponding to seasonal variation, this model generally does not capture the shape of the data or minimum/maximum values well. Only the first principal component was retained for simplicity of the model, but to improve the predictions more principal components should be considered. Additional principal components may more accurately capture heavy precipitation patterns and predict values closer to actual precipitation on heavy rainfall days.

A limitation of both models is the handling of missing precipitation values. This analysis elected to replace missing values with 0.0, but this is likely altering predictive accuracy by introducing bias. An improvement would be to remove missing values altogether before conducting data analysis. Another major limitation of both models is only using temperature data to predict precipitation. Based on prior knowledge, pressure data is also important in predicting precipitation and model performance likely would have significantly included if more variables beyond temperature were included.


In [ ]:
mae_lin_mean = mean(mae_lin)
mae_knn_mean = mean(mae_knn) 
mse_knn_mean = mean(mse_knn)
mse_lin_mean = mean(mse_lin)
residuals_lin_mean = mean(residuals_lin)
residuals_knn_mean = mean(residuals_knn)

In [ ]:
#create bar chart mae
bar(["Linear Regression", "KNN"], [mae_lin_mean, mae_knn_mean], label="MAE", title="MAE for Linear Regression and KNN", xlabel="Model", ylabel="MAE", legend=:topleft)

In [ ]:
#create bar chart mse
bar(["Linear Regression", "KNN"], [mse_lin_mean, mse_knn_mean], label="MSE", title="MSE for Linear Regression and KNN", xlabel="Model", ylabel="MSE", legend=:topleft)

In [ ]:
#create bar chart residuals
bar(["Linear Regression", "KNN"], [residuals_lin_mean, residuals_knn_mean], label="Residuals", title="Residuals for Linear Regression and KNN", xlabel="Model", ylabel="Residuals", legend=:topleft)

7. Conclusion

This report provides preliminary models to predict precipitation at time t+1 based on temperature at time t over Texas. Both approaches employ a principal component analysis to downscale high dimensional spatial data while retaining variance explained within the data. Approach 1 then applies K-nearest neighbors to predict precipitation using weighted sampling indexes of the three closest neighbors. After PCA, Approach 2 applies a linear regression to the first principal component at each grid cell of precipitation following temperature. When comparing predicted and actual precipitation across the two models, Approach 1 appears to capture the range of precipitation values more accurately but exhibits a higher mean squared error and mean absolute error, indicating poorer overall model performance. Approach 2 predicts low level precipitation across the entire time series and thus has lower mean squared error and mean absolute error values due to a lack of appropriate extremes.

If the goal of the model prediction was to minimize MSE and MAE, then Approach 2 would be a better model. Both models would be significantly improved by the optimization of chosen model parameters and the inclusion of additional climate variables, including pressure, to predict precipitation. 